# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
os.listdir()

['train',
 'val',
 '6.+Cifar10_morelayer_notebook.ipynb',
 '3.+Cifar_10_notebook.ipynb',
 '7.+Cifar10_feature_map.ipynb',
 '1.+Cifar_10_with_dropout_without_BN.ipynb',
 'val.csv',
 '.ipynb_checkpoints',
 'train.csv',
 '2.+Cifar_10_Notebook_with_BN_without_dropout.ipynb',
 'Neural_Nets_Project_Starter_Code.ipynb',
 '4.+Cifar10_l2_notebook.ipynb',
 '5.+Cifar10_l2_dropout_notebook.ipynb']

In [58]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size = 30

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [59]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [rn.randint(0,30) for j in range(0,20)]
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,20,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    if image.shape[0] != image.shape[1]:
                        image = image[0:120,20:140]
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized = imresize(image,(120,120,3), interp='bilinear')
                    # normal data
                    batch_data[folder,idx] = (resized)
                    batch_data[folder,idx,:,:,0] = (resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (resized[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (resized[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
    # write the code for the remaining data points which are left after full batches
    if (len(t)%batch_size) != 0:
        remaining_batch = len(t)%batch_size
        batch_data = np.zeros((remaining_batch,20,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((remaining_batch,5)) # batch_labels is the one hot representation of the output
        for folder in range(remaining_batch): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*remaining_batch)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (batch*remaining_batch)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape 
                if image.shape[0] != image.shape[1]:
                    image = image[0:120,20:140]
                #and the conv3D will throw error if the inputs in a batch have different shapes
                resized = imresize(image,(120,120,3), interp='bilinear')
                # normal data
                batch_data[folder,idx] = (resized)
                batch_data[folder,idx,:,:,0] = (resized[:,:,0])/255 #normalise and feed in the image
                batch_data[folder,idx,:,:,1] = (resized[:,:,1])/255 #normalise and feed in the image
                batch_data[folder,idx,:,:,2] = (resized[:,:,2])/255 #normalise and feed in the image

            batch_labels[folder, int(t[folder + (batch*remaining_batch)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [60]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [61]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

#write your model here
# model = Sequential()

vgg = VGG16(include_top=False, weights='imagenet', input_shape=(120,120,3))

for layer in vgg.layers:
    layer.trainable = False # Create Sequential Model

model = Sequential()
model.add(TimeDistributed(vgg,input_shape=(20,120,120,3)))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.3))
model.add(GRU(256,return_sequences=False,dropout=0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [62]:
optimiser = optimizers.Nadam(lr=0.002,beta_1=0.9,beta_2=0.999,epsilon=1e-08,schedule_decay=0.004)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 20, 3, 3, 512)     14714688  
_________________________________________________________________
time_distributed_20 (TimeDis (None, 20, 1, 1, 512)     0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 20, 512)           0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 20, 512)           0         
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               590592    
_________________________________________________________________
dense_19 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [63]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [64]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=2,verbose=1,mode='auto',epsilon=1e-04,cooldown=1,min_lr=0) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [65]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [66]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1,
                    initial_epoch=0)

Source path =  val Source path =  train ; batch size = 30
; batch size = 30
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 133s 6s/step - loss: 1.6477 - categorical_accuracy: 0.2261 - val_loss: 1.4636 - val_categorical_accuracy: 0.4083

Epoch 00001: val_loss improved from inf to 1.46359, saving model to model_init_2020-04-1312_54_22.586719/model-00001-1.64771-0.22609-1.46359-0.40833.h5
Epoch 2/20
23/23 [==============================] - 55s 2s/step - loss: 1.4855 - categorical_accuracy: 0.3652 - val_loss: 1.3493 - val_categorical_accuracy: 0.4083

Epoch 00002: val_loss improved from 1.46359 to 1.34927, saving model to model_init_2020-04-1312_54_22.586719/model-00002-1.48547-0.36522-1.34927-0.40833.h5
Epoch 3/20
23/23 [==============================] - 54s 2s/step - loss: 1.3456 - categorical_accuracy: 0.4478 - val_loss: 1.1900 - val_categorical_accuracy: 0.5333

Epoch 00003: val_loss improved from 1.34927 to 1.18997, saving model to model_init_2020-04-1312_54_22.586719/model-00003-1.34563-0.44783-1.18997-0.53333.h5
Epoch 4/20
23/23 [==============================] -